In [ ]:
!pip install pydicom
!pip install chart_studio

     |████████████████████████████████| 1.9MB 17.3MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 


In [ ]:
!mkdir AllResults_HGSR
!mkdir result_files_HGSR
!mkdir Model_HGSR

In [ ]:
import numpy as np
import os, sys, stat
import copy
from math import *
import matplotlib.pyplot as plt
from functools import reduce
# reading in dicom files
import pydicom
# skimage image processing packages
# from skimage import measure, morphology
# from skimage.morphology import ball, binary_closing
# from skimage.measure import label, regionprops
# scipy linear algebra functions
# from scipy.linalg import norm
import scipy.ndimage
# ipywidgets for some interactive plots
from ipywidgets.widgets import *
import ipywidgets as widgets
# plotly 3D interactive graphs
import plotly
from plotly.graph_objs import *
import chart_studio.plotly as py
import cv2
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
import scipy.misc
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import concatenate, Add
from keras import backend as K
import math
import imageio
from contextlib import redirect_stdout
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from keras.layers.core import Dropout, Lambda


# Set some parameters
IMG_WIDTH = 480
IMG_HEIGHT = 480
IMG_CHANNELS = 3

In [ ]:
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2),padding='same') (c1)

c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(3, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 480, 480, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 480, 480, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 480, 480, 16) 448         lambda[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 480, 480, 16) 0           conv2d[0][0]                     
______________________________________________________________________________________________

In [ ]:
def test(model):
    z = range(0, 18)
    sliced_z = 18
    test_T1 = "/content/drive/MyDrive/d2 new/test/T1"
    test_T2 = "/content/drive/MyDrive/d2 new/test/T2"
    folder_3 = os.listdir(test_T1)
    folder_4 = os.listdir(test_T2)
    T1_test_matrix = []
    T2_test_matrix = []
    T1x_test_images = []
    T2x_test_images = []
    T1y_test_images = []
    T2y_test_images = []

    for f in folder_3:
        T1xx_images = np.zeros([480,480,18]) 
        T1yx_images = np.zeros([480,480,18]) 
        temp = np.zeros([480,480,18])
        path = test_T1 + '/' + f
        slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key=lambda x: int(x.ImagePositionPatient[2]))
       
        pixel_spacing = slices[0].PixelSpacing   
        slice_thickness = slices[0].SliceThickness

        # print(pixel_spacing)
        # print(slice_thickness)
    
        image_shape = list(slices[0].pixel_array.shape)
        image_shape.append(len(slices))

        volume3d = np.zeros(image_shape)
    
        for i,s in enumerate(slices):
            array2d = s.pixel_array
            volume3d[:,:,i] = array2d
        a = volume3d
        
        temp[3:, 3:, :] = a[3:480, 3:480, z]
        a = temp
        for k in range(sliced_z):
          T1xx_images[:,:,k]=cv2.Sobel(a[:,:,k],cv2.CV_64F,1,0,ksize=5)
          T1yx_images[:,:,k]=cv2.Sobel(a[:,:,k],cv2.CV_64F,0,1,ksize=5)
        T1xx_images = np.asarray(T1xx_images)
        T1yx_images = np.asarray(T1yx_images)
        img_vol_min = np.min(a)
        img_vol_max = np.max(a)
        img_vol_x_min = np.min(T1xx_images)
        img_vol_x_max = np.max(T1xx_images)
        img_vol_y_min = np.min(T1yx_images)
        img_vol_y_max = np.max(T1yx_images)

        a = (a - img_vol_min) / (img_vol_max - img_vol_min)
        # T1xx_images = (T1xx_images - img_vol_x_min) / (img_vol_x_max - img_vol_x_min)
        # T1yx_images = (T1yx_images - img_vol_y_min) / (img_vol_y_max - img_vol_y_min)

        for some in range(sliced_z):
            T1_test_matrix.append(a[:, :, some])
            T1x_test_images.append(T1xx_images[:, :, some])
            T1y_test_images.append(T1yx_images[:, :, some])
            # del(T1xx_images)
            # del(T1yx_images)
            
            
    for f in folder_4:
        T2xx_images = np.zeros([480,480,18]) 
        T2yx_images = np.zeros([480,480,18]) 
        temp = np.zeros([480,480,18])
        path = test_T2 + '/' + f
        slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key=lambda x: int(x.ImagePositionPatient[2]))
        pixel_spacing = slices[0].PixelSpacing   
        slice_thickness = slices[0].SliceThickness

        # print(pixel_spacing)
        # print(slice_thickness)
    
        image_shape = list(slices[0].pixel_array.shape)
        image_shape.append(len(slices))
        volume3d = np.zeros([480,480,18])
        # volume3d = np.zeros(image_shape)
    
        for i,s in enumerate(slices):
            array2d = s.pixel_array
            resized_img = resize(array2d, (480, 480), anti_aliasing=True)
            volume3d[:,:,i] = resized_img
        b = volume3d
        temp[3:, 3:, :] = b[3:480, 3:480, z]
        b = temp
        for k in range(sliced_z):
          T2xx_images[:,:,k]=cv2.Sobel(b[:,:,k],cv2.CV_64F,1,0,ksize=5)
          T2yx_images[:,:,k]=cv2.Sobel(b[:,:,k],cv2.CV_64F,0,1,ksize=5)

        T2xx_images = np.asarray(T2xx_images)
        T2yx_images = np.asarray(T2yx_images)
        img_vol_min = np.min(b)
        img_vol_max = np.max(b)
        img_vol_x_min = np.min(T2xx_images)
        img_vol_x_max = np.max(T2xx_images)
        img_vol_y_min = np.min(T2yx_images)
        img_vol_y_max = np.max(T2yx_images)
        
        b = (b - img_vol_min) / (img_vol_max - img_vol_min)
        # T2xx_images = (T2xx_images - img_vol_x_min) / (img_vol_x_max - img_vol_x_min)
        # T2yx_images = (T2yx_images - img_vol_y_min) / (img_vol_y_max - img_vol_y_min)

        for some in range(sliced_z):
            T2_test_matrix.append(b[:, :, some])
            T2x_test_images.append(T2xx_images[:, :, some])
            T2y_test_images.append(T2yx_images[:, :, some])

        del(T2xx_images)
        del(T2yx_images)

    print("read both T1_test and T2_test")

    # T1_test_matrix.shape (51 * number of sucjects in T1_registered, 259, 259)
    T1_test_matrix = np.asarray(T1_test_matrix)
    # T2_test_matrix.shape (51 * number of sucjects in ground_T2, 259, 259)
    T2_test_matrix = np.asarray(T2_test_matrix)
    # T1x_test_images.shape (51 * number of sucjects in T1_registered, 259, 259)
    T1x_test_images = np.asarray(T1x_test_images)
    # T2x_images.shape (51 * number of sucjects in ground_T2, 259, 259)
    T2x_test_images = np.asarray(T2x_test_images)
    # T1y_images.shape (51 * number of sucjects in T1_registered, 259, 259)
    T1y_test_images = np.asarray(T1y_test_images)
    # T2y_images.shape (51 * number of sucjects in ground_T2, 259, 259)
    T2y_test_images = np.asarray(T2y_test_images)

    T1_test_data = np.zeros(shape=[(T1_test_matrix.shape[0]), (T1_test_matrix.shape[1]), (T1_test_matrix.shape[2]), (3)])
    T2_ground_truth = np.zeros(shape=[(T1_test_matrix.shape[0]), (T1_test_matrix.shape[1]), (T1_test_matrix.shape[2]), (3)])

    # resizeTo = 256
    for i in range(T1_test_matrix.shape[0]):
        T1_test_data[i, :, :, 0] = T1_test_matrix[i, :, :]
        T1_test_data[i, :, :, 1] = T1x_test_images[i, :, :]
        T1_test_data[i, :, :, 2] = T1y_test_images[i, :, :]
        T2_ground_truth[i, :, :, 0] = T2_test_matrix[i, :, :]
        T2_ground_truth[i, :, :, 1] = T2x_test_images[i, :, :]
        T2_ground_truth[i, :, :, 2] = T2y_test_images[i, :, :]

    del(T1_test_matrix)
    del(T1x_test_images)
    del(T1y_test_images)
    del(T2_test_matrix)
    del(T2x_test_images)
    del(T2y_test_images)
    print("made all the augmented images")

    # values less than a particular value has been made 0
    z, x, y = np.where(T1_test_data[:, :, :, 0] < 0.1)
    T1_test_data[z, x, y, 0] = 0
    z, x, y = np.where(T2_ground_truth[:, :, :, 0] < 0.1)
    T2_ground_truth[z, x, y, 0] = 0
    z, x, y = np.where(T1_test_data[:, :, :, 1] < 0.001)
    T1_test_data[z, x, y, 1] = 0
    z, x, y = np.where(T2_ground_truth[:, :, :, 1] < 0.001)
    T2_ground_truth[z, x, y, 1] = 0
    z, x, y = np.where(T1_test_data[:, :, :, 2] < 0.001)
    T1_test_data[z, x, y, 2] = 0
    z, x, y = np.where(T2_ground_truth[:, :, :, 2] < 0.001)
    T2_ground_truth[z, x, y, 2] = 0

    T1_test_data, T2_ground_truth = shuffle(T1_test_data, T2_ground_truth, random_state=2)

    myfile_test_psnr_T2_0 = open('/content/result_files_HGSR/test_psnrT2_0.txt', 'a')
    myfile_test_mse_T2_0 = open('/content/result_files_HGSR/test_mseT2_0.txt', 'a')
    myfile_test_mae_T2_0 = open('/content/result_files_HGSR/test_maeT2_0.txt', 'a')

    myfile_test_psnr_T2_1 = open('/content/result_files_HGSR/test_psnrT2_1.txt', 'a')
    myfile_test_mse_T2_1 = open('/content/result_files_HGSR/test_mseT2_1.txt', 'a')
    myfile_test_mae_T2_1 = open('/content/result_files_HGSR/test_maeT2_1.txt', 'a')

    myfile_test_psnr_T2_2 = open('/content/result_files_HGSR/test_psnrT2_2.txt', 'a')
    myfile_test_mse_T2_2 = open('/content/result_files_HGSR/test_mseT2_2.txt', 'a')
    myfile_test_mae_T2_2 = open('/content/result_files_HGSR/test_maeT2_2.txt', 'a')

    test_all_imgs_mse_file = open('/content/result_files_HGSR/test_all_imgs_mse_file.txt', 'a')
    test_all_imgs_mae_file = open('/content/result_files_HGSR/test_all_imgs_mae_file.txt', 'a')
    test_all_imgs_psnr_file = open('/content/result_files_HGSR/test_all_imgs_psnr_file.txt', 'a')

    decoded_imgs = model.predict(T1_test_data)

# edit 3
    mse_T2_0 = np.mean((T2_ground_truth[:, :, :, 0] - decoded_imgs[:, :, :, 0]) ** 2)
    check_T2_0 = math.sqrt(mse_T2_0)
    psnr_T2_0 = 20 * math.log10(1.0 / check_T2_0)
    mae_T2_0 = np.mean(np.abs((T2_ground_truth[:, :, :, 0] - decoded_imgs[:, :, :, 0])))

    mse_T2_1 = np.mean(
        (T2_ground_truth[:, :, :, 1] - decoded_imgs[:, :, :, 1]) ** 2)
    check_T2_1 = math.sqrt(mse_T2_1)
    psnr_T2_1 = 20 * math.log10(1.0 / check_T2_1)
    mae_T2_1 = np.mean(np.abs((T2_ground_truth[:, :, :, 1] - decoded_imgs[:, :, :, 1])))

    mse_T2_2 = np.mean((T2_ground_truth[:, :, :, 2] - decoded_imgs[:, :, :, 2]) ** 2)
    check_T2_2 = math.sqrt(mse_T2_2)
    psnr_T2_2 = 20 * math.log10(1.0 / check_T2_2)
    mae_T2_2 = np.mean(np.abs((T2_ground_truth[:, :, :, 2] - decoded_imgs[:, :, :, 2])))

    myfile_test_psnr_T2_0.write("%f \n" % (psnr_T2_0))
    myfile_test_mse_T2_0.write("%f \n" % (mse_T2_0))
    myfile_test_mae_T2_0.write("%f \n" % (mae_T2_0))

    myfile_test_psnr_T2_1.write("%f \n" % (psnr_T2_1))
    myfile_test_mse_T2_1.write("%f \n" % (mse_T2_1))
    myfile_test_mae_T2_1.write("%f \n" % (mae_T2_1))

    myfile_test_psnr_T2_2.write("%f \n" % (psnr_T2_2))
    myfile_test_mse_T2_2.write("%f \n" % (mse_T2_2))
    myfile_test_mae_T2_2.write("%f \n" % (mae_T2_2))

    test_all_imgs_mse = np.mean((T2_ground_truth - decoded_imgs) ** 2)
    test_all_imgs_mae = np.mean(np.abs(T2_ground_truth - decoded_imgs))
    check = math.sqrt(test_all_imgs_mse)
    test_all_imgs_psnr = 20 * math.log10(1.0 / check)
    print("mse of all images in test set is")
    print(test_all_imgs_mse)
    print("mae of all images in test set is")
    print(test_all_imgs_mae)
    print("psnr of all images in test set is")
    print(test_all_imgs_psnr)

    test_all_imgs_mse_file.write("%f \n" % test_all_imgs_mse)
    test_all_imgs_mae_file.write("%f \n" % test_all_imgs_mae)
    test_all_imgs_psnr_file.write("%f \n" % test_all_imgs_psnr)

    resizeTo = 480
    temp = np.zeros([1440, 1440])
    print("Shape of T1_test_data")
    print(T1_test_data.shape)
    print(type(T1_test_data))
    print("Shape of temp")
    print(temp.shape)
    print(type(temp))
    temp[:480, :480] = T1_test_data[0, :, :, 0]
    temp[480:480*2, :480] = T1_test_data[0, :, :, 1]
    temp[resizeTo*2:, :resizeTo] = T1_test_data[0, :, :, 2]
    temp[:resizeTo, resizeTo:resizeTo*2] = T2_ground_truth[0, :, :, 0]
    temp[resizeTo:resizeTo*2, resizeTo:resizeTo*2] = T2_ground_truth[0, :, :, 1]
    temp[resizeTo*2:, resizeTo:resizeTo*2] = T2_ground_truth[0, :, :, 2]
    temp[:resizeTo, 2*resizeTo:] = decoded_imgs[0, :, :, 0]
    temp[resizeTo:resizeTo*2, 2*resizeTo:] = decoded_imgs[0, :, :, 1]
    temp[resizeTo*2:, 2*resizeTo:] = decoded_imgs[0, :, :, 2]
    temp = temp * 480
    # scipy.misc.imsave('results_sliced_4layers_custom_loss/' + str(jj) + '.jpg', temp)
    imageio.imwrite('/content/AllResults_HGSR/test' + '.jpg', temp.astype(np.uint8))
    
    myfile_test_psnr_T2_0.close()
    myfile_test_mse_T2_0.close()
    myfile_test_mae_T2_0.close()

    myfile_test_psnr_T2_1.close()
    myfile_test_mse_T2_1.close()
    myfile_test_mae_T2_1.close()

    myfile_test_psnr_T2_2.close()
    myfile_test_mse_T2_2.close()
    myfile_test_mae_T2_2.close()

    test_all_imgs_mse_file.close()
    test_all_imgs_mae_file.close()
    test_all_imgs_psnr_file.close()

i_s = 480
# input_shape = Input(shape = (480, 480, 3))#tensor called input_shape is done
# gen = gen(input_shape)
# proSR = net2(input_shape)
# output = Add()([gen,proSR])
# model = Model(input_shape, output)
# #model.load_weights("Model_HGSR/model_epoch_295.h5") #for loading the saved weights, prefix this line before compiling
# model.compile(loss=root_mean_sq_GxGy, optimizer = RMSprop())
#print(model.summary())
#os.chmod("E:/FYP DATA/hourglass_sr.txt", stat.S_IwRITE)
 
with open('/content/hourglass_sr.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

#x, y = 256, 256

z = range(0,18)
sliced_z = 18


#resizeTo = 259
#batch_size = 8
#inChannel = outChannel = 3

train_T1 = "/content/drive/MyDrive/d2 new/train/T1"
train_T2 = "/content/drive/MyDrive/d2 new/train/T2"
folder_1 = os.listdir(train_T1)
folder_2 = os.listdir(train_T2)


T1_train_matrix = []
T2_train_matrix = []
T1x_images = []
T2x_images = []
T1y_images = []
T2y_images = []


for f in folder_1:
    path = train_T1 + '/' + f
    T1xx_images = np.zeros([480,480,18])    
    T1yx_images = np.zeros([480,480,18])
    
    temp = np.zeros([480,480,18])    
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.ImagePositionPatient[2]))
    pixel_spacing = slices[0].PixelSpacing   
    slice_thickness = slices[0].SliceThickness

    # print(pixel_spacing)
    # print(slice_thickness)
    
    image_shape = list(slices[0].pixel_array.shape)
    image_shape.append(len(slices))

    volume3d = np.zeros(image_shape)
    
    for i,s in enumerate(slices):
        array2d = s.pixel_array
        volume3d[:,:,i] = array2d
    a = volume3d
    temp[3:, 3:, :] = a[3:480, 3:480, z]
    a = temp
    print(a.shape)
    
    for k in range(sliced_z):
        T1xx_images[:,:,k]=cv2.Sobel(a[:,:,k],cv2.CV_64F,1,0,ksize=5)
        T1yx_images[:,:,k]=cv2.Sobel(a[:,:,k],cv2.CV_64F,0,1,ksize=5)
    T1xx_images=np.asarray(T1xx_images)
    # print('after 549 line t1xx')
    print(T1xx_images.shape)
    T1yx_images=np.asarray(T1yx_images)
    img_vol_min = np.min(a)
    img_vol_max = np.max(a)
    img_vol_x_min = np.min(T1xx_images)
    img_vol_x_max = np.max(T1xx_images)
    img_vol_y_min = np.min(T1yx_images)
    img_vol_y_max = np.max(T1yx_images)

    #Volume normaization of each image.
    a = (a - img_vol_min) / (img_vol_max - img_vol_min)
#    print('a shape')
    print(a.shape)
    #a.shape == (img_row, slices, img_col : 259, 51, 259)
    # T1xx_images = (T1xx_images - img_vol_x_min) / (img_vol_x_max - img_vol_x_min)
    # print('txx shape')
    # print(T1xx_images.shape)#T1xx_images.shape = (slices, img_row, img_col : 51, 259, 259)
    # T1yx_images = (T1yx_images - img_vol_y_min) / (img_vol_y_max - img_vol_y_min) #T1yx_images.shape = (slices, img_row, img_col : 51, 259, 259)
    
    for some in range(sliced_z):
        T1_train_matrix.append(a[:,:,some]) #T1_train_matrix.shape is made into (51, 259, 259) from a.shape which was (259, 51, 259)
        T1x_images.append(T1xx_images[:, :, some])
        T1y_images.append(T1yx_images[:, :, some])
        #All there arrays have same shape (51, 259, 259)
    
    # del(T1xx_images)
    # del(T1yx_images)
    
    
for f in folder_2:
    path = train_T2 + '/' + f
    T2xx_images = np.zeros([480,480,18])  
    T2yx_images = np.zeros([480,480,18])  
    
    temp = np.zeros([480,480,18])
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.ImagePositionPatient[2]))
    pixel_spacing = slices[0].PixelSpacing   
    slice_thickness = slices[0].SliceThickness

    # print(pixel_spacing)
    # print(slice_thickness)
    
    image_shape = list(slices[0].pixel_array.shape)
    image_shape.append(len(slices))
    volume3d = np.zeros([480,480,18])
    # volume3d = np.zeros(image_shape)
    
    for i,s in enumerate(slices):
        array2d = s.pixel_array
        resized_img = resize(array2d, (480, 480), anti_aliasing=True)
        volume3d[:,:,i] = resized_img
    b = volume3d
    
    temp[3:, 3:, :] = b[3:480, 3:480, z]
    b = temp
    
    for k in range(sliced_z):
        T2xx_images[:,:,k]=cv2.Sobel(b[:,:,k],cv2.CV_64F,1,0,ksize=5)
        T2yx_images[:,:,k]=cv2.Sobel(b[:,:,k],cv2.CV_64F,0,1,ksize=5)
    T2xx_images=np.asarray(T2xx_images)
    T2yx_images=np.asarray(T2yx_images)
    img_vol_min = np.min(b)
    img_vol_max = np.max(b)
    img_vol_x_min = np.min(T2xx_images)
    img_vol_x_max = np.max(T2xx_images)
    img_vol_y_min = np.min(T2yx_images)
    img_vol_y_max = np.max(T2yx_images)

    #Volume normaization of each image.
    b = (b - img_vol_min) / (img_vol_max - img_vol_min)
    # T2xx_images = (T2xx_images - img_vol_x_min) / (img_vol_x_max - img_vol_x_min)
    # T2yx_images = (T2yx_images - img_vol_y_min) / (img_vol_y_max - img_vol_y_min)
    
    for some in range(sliced_z):
        T2_train_matrix.append(b[:,:,some])
        T2x_images.append(T2xx_images[:, :, some])
        T2y_images.append(T2yx_images[:, :, some])
    
    del(T2xx_images)
    del(T2yx_images)

print("read both ground and input")


#T1_train_matrix = np.zeros([])
T1_train_matrix = np.asarray(T1_train_matrix) #T1_train_matrix.shape (51 * number of sucjects in T1_registered, 259, 259)
T2_train_matrix = np.asarray(T2_train_matrix) #T2_train_matrix.shape (51 * number of sucjects in ground_T2, 259, 259)
T1x_images = np.asarray(T1x_images) #T1x_images.shape (51 * number of sucjects in T1_registered, 259, 259)
T2x_images = np.asarray(T2x_images) #T2x_images.shape (51 * number of sucjects in ground_T2, 259, 259)
T1y_images = np.asarray(T1y_images) #T1y_images.shape (51 * number of sucjects in T1_registered, 259, 259)
T2y_images = np.asarray(T2y_images) #T2y_images.shape (51 * number of sucjects in ground_T2, 259, 259)


data=np.zeros(shape=[(T1_train_matrix.shape[0]),(T1_train_matrix.shape[1]),(T1_train_matrix.shape[2]),(3)])
Label=np.zeros(shape=[(T1_train_matrix.shape[0]),(T1_train_matrix.shape[1]),(T1_train_matrix.shape[2]),(3)])
#data will have shape (51 * number of subjects in T1_registered, 259, 259, 3 (for image, its x gradient, its y gradient))
#Label will have shape (51 * number of subjects in ground_T2, 259, 259, 3 (for image, its x gradient, its y gradient))

#resizeTo = 256
for i in range(T1_train_matrix.shape[0]):
    data[i,:,:,0] = T1_train_matrix[i,:,:]
    data[i,:,:,1] = T1x_images[i,:,:]
    data[i,:,:,2] = T1y_images[i,:,:]
    Label[i,:,:,0] = T2_train_matrix[i,:,:]
    Label[i,:,:,1] = T2x_images[i,:,:]
    Label[i,:,:,2] = T2y_images[i,:,:]
    
del(T1_train_matrix)
del(T1x_images)
del(T1y_images)
del(T2_train_matrix)
del(T2x_images)
del(T2y_images)
print("made all the augmented images")

#values less than a particular value has been made 0
z,x,y=np.where(data[:,:,:,0]<0.1)    
data[z,x,y,0]=0
z,x,y=np.where(Label[:,:,:,0]<0.1)    
Label[z,x,y,0]=0
z,x,y=np.where(data[:,:,:,1]<0.001)    
data[z,x,y,1]=0
z,x,y=np.where(Label[:,:,:,1]<0.001)    
Label[z,x,y,1]=0
z,x,y=np.where(data[:,:,:,2]<0.001)    
data[z,x,y,2]=0
z,x,y=np.where(Label[:,:,:,2]<0.001)    
Label[z,x,y,2]=0

data, Label = shuffle(data, Label, random_state=2)
print(data.shape)
print('printing label shape below')
print(Label.shape)
train_T1, validation_T1, train_T2, validation_T2 = train_test_split(data, Label, test_size=0.2, random_state=2)

print("Training and Testing data has now following shapes")
print(train_T1.shape)
print(validation_T1.shape)

#del(data)
del(Label)

print("Training data lenth is : ", len(train_T1))

validation_T1 = np.array(validation_T1)

validation_T2 = np.array(validation_T2)

train_T1 = np.array(train_T1)
train_T2 = np.array(train_T2)

resizeTo = 480

numEpochs = 150
batch_size = 2


for jj in range(0,numEpochs):
    print("Running epoch : %d" % jj)
    myfile_valid_psnr_T2_0 = open('/content/result_files_HGSR/validation_psnrT2_0.txt', 'w+')
    myfile_valid_mse_T2_0 = open('/content/result_files_HGSR/validation_mseT2_0.txt', 'a+')
    myfile_valid_psnr_dat_T2_0 = open('/content/result_files_HGSR/validation_psnrT2_0.dat', 'a+')   #can be removed
    myfile_valid_mse_dat_T2_0 = open('/content/result_files_HGSR/validation_mseT2_0.dat', 'a+')
    myfile_valid_mae_T2_0 = open('/content/result_files_HGSR/validation_maeT2_0.txt', 'a+')
    
    myfile_valid_psnr_T1_0 = open('/content/result_files_HGSR/validation_psnrT1_0.txt', 'a+')
    myfile_valid_mse_T1_0 = open('/content/result_files_HGSR/validation_mseT1_0.txt', 'a+')
    myfile_valid_psnr_dat_T1_0 = open('/content/result_files_HGSR/validation_psnrT1_0.dat', 'a+')
    myfile_valid_mse_dat_T1_0 = open('/content/result_files_HGSR/validation_mseT1_0.dat', 'a+')

    myfile_valid_psnr_T2_1 = open('/content/result_files_HGSR/validation_psnrT2_1.txt', 'a+')
    myfile_valid_mse_T2_1 = open('/content/result_files_HGSR/validation_mseT2_1.txt', 'a+')
    myfile_valid_psnr_dat_T2_1 = open('/content/result_files_HGSR/validation_psnrT2_1.dat', 'a+')
    myfile_valid_mse_dat_T2_1 = open('/content/result_files_HGSR/validation_mseT2_1.dat', 'a+')
    myfile_valid_mae_T2_1 = open('/content/result_files_HGSR/validation_maeT2_1.txt', 'a+')
    
    myfile_valid_psnr_T1_1 = open('/content/result_files_HGSR/validation_psnrT1_1.txt', 'a+')
    myfile_valid_mse_T1_1 = open('/content/result_files_HGSR/validation_mseT1_1.txt', 'a+')
    myfile_valid_psnr_dat_T1_1 = open('/content/result_files_HGSR/validation_psnrT1_1.dat', 'a+')
    myfile_valid_mse_dat_T1_1 = open('/content/result_files_HGSR/validation_mseT1_1.dat', 'a+')

    myfile_valid_psnr_T2_2 = open('/content/result_files_HGSR/validation_psnrT2_2.txt', 'a+')
    myfile_valid_mse_T2_2 = open('/content/result_files_HGSR/validation_mseT2_2.txt', 'a+')
    myfile_valid_psnr_dat_T2_2 = open('/content/result_files_HGSR/validation_psnrT2_2.dat', 'a+')
    myfile_valid_mse_dat_T2_2 = open('/content/result_files_HGSR/validation_mseT2_2.dat', 'a+')
    myfile_valid_mae_T2_2 = open('/content/result_files_HGSR/validation_maeT2_2.txt', 'a+')
    
    myfile_valid_psnr_T1_2 = open('/content/result_files_HGSR/validation_psnrT1_2.txt', 'a+')
    myfile_valid_mse_T1_2 = open('/content/result_files_HGSR/validation_mseT1_2.txt', 'a+')
    myfile_valid_psnr_dat_T1_2 = open('/content/result_files_HGSR/validation_psnrT1_2.dat', 'a+')
    myfile_valid_mse_dat_T1_2 = open('/content/result_files_HGSR/validation_mseT1_2.dat', 'a+')

    batch_loss_file = open('/content/result_files_HGSR/batch_loss_file.txt', 'a+')
    valid_all_imgs_mse_file = open('/content/result_files_HGSR/valid_all_imgs_mse_file.txt', 'a+')
    valid_all_imgs_mae_file = open('/content/result_files_HGSR/valid_all_imgs_mae_file.txt', 'a+')
    valid_all_imgs_psnr_file = open('/content/result_files_HGSR/valid_all_imgs_psnr_file.txt', 'a+')
    train_T1, train_T2 = shuffle(train_T1, train_T2)
    num_batches = int(len(train_T1)/batch_size)
    # num_batches = 15
    for batch in range(num_batches):

        batch_train_X = train_T1[batch*batch_size:min((batch+1)*batch_size,len(train_T1)),:]
        batch_train_Y = train_T2[batch*batch_size:min((batch+1)*batch_size,len(train_T2)),:]
        
        loss = model.train_on_batch(batch_train_X,batch_train_Y)
        #print ('epoch_num: %d batch_num: %d loss: %f\n' % (jj,batch,loss))
        #batch_loss_file.write("%d %d %f\n" % (jj, batch, loss))

    #default_loss.append(loss)
    model.save_weights("/content/Model_HGSR/last_model.h5")
    if jj > 290:
        model.save_weights("/content/Model_HGSR/model_epoch_"+ str(jj) +".h5")
    #validation_T1,validation_T2 = shuffle(validation_T1,validation_T2)
    #validation_loss = autoencoder.test_on_batch(X_test,y_test)
    print("The shape is")
    decoded_imgs = model.predict(validation_T1)
    imageio.imwrite("decoded.png", decoded_imgs[0,:,:,0].astype(np.uint8))
    print(decoded_imgs.shape)
    print(validation_T2[:,:,:,0].shape)

    mse_T2_0 =  np.mean((validation_T2[:,:,:,0] - decoded_imgs[:,:,:,0]) ** 2)
    check_T2_0 = math.sqrt(mse_T2_0)
    psnr_T2_0 = 20 * math.log10( 1.0 / check_T2_0)
    mae_T2_0 = np.mean(np.abs((validation_T2[:,:,:,0] - decoded_imgs[:,:,:,0])))


    mse_T1_0=  np.mean((validation_T1[:,:,:,0] - decoded_imgs[:,:,:,0]) ** 2)
    check_T1_0 = math.sqrt(mse_T1_0)
    psnr_T1_0 = 20 * math.log10( 1.0 / check_T1_0)

    mse_T2_1=  np.mean((validation_T2[:,:,:,1] - decoded_imgs[:,:,:,1]) ** 2)
    check_T2_1 = math.sqrt(mse_T2_1)
    psnr_T2_1 = 20 * math.log10( 1.0 / check_T2_1)
    mse_T1_1 =  np.mean((validation_T1[:,:,:,1] - decoded_imgs[:,:,:,1]) ** 2)
    check_T1_1 = math.sqrt(mse_T1_1)
    psnr_T1_1 = 20 * math.log10( 1.0 / check_T1_1)
    mae_T2_1 = np.mean(np.abs((validation_T2[:,:,:,1] - decoded_imgs[:,:,:,1])))

    mse_T2_2 =  np.mean((validation_T2[:,:,:,2] - decoded_imgs[:,:,:,2]) ** 2)
    check_T2_2 = math.sqrt(mse_T2_2)
    psnr_T2_2 = 20 * math.log10( 1.0 / check_T2_2)
    mse_T1_2 =  np.mean((validation_T1[:,:,:,2] - decoded_imgs[:,:,:,2]) ** 2)
    check_T1_2 = math.sqrt(mse_T1_2)
    psnr_T1_2 = 20 * math.log10( 1.0 / check_T1_2)
    mae_T2_2 = np.mean(np.abs((validation_T2[:,:,:,2] - decoded_imgs[:,:,:,2])))

    myfile_valid_psnr_T2_0.write("%f \n" % (psnr_T2_0))
    myfile_valid_mse_T2_0.write("%f \n" % (mse_T2_0))
    myfile_valid_psnr_dat_T2_0.write("%f \n" % (psnr_T2_0))
    myfile_valid_mse_dat_T2_0.write("%f \n" % (mse_T2_0))
    myfile_valid_mae_T2_0.write("%f \n" %(mae_T2_0))

    myfile_valid_psnr_T1_0.write("%f \n" % (psnr_T1_0))
    myfile_valid_mse_T1_0.write("%f \n" % (mse_T1_0))
    myfile_valid_psnr_dat_T1_0.write("%f \n" % (psnr_T1_0))
    myfile_valid_mse_dat_T1_0.write("%f \n" % (mse_T1_0))
    
    myfile_valid_psnr_T2_1.write("%f \n" % (psnr_T2_1))
    myfile_valid_mse_T2_1.write("%f \n" % (mse_T2_1))
    myfile_valid_psnr_dat_T2_1.write("%f \n" % (psnr_T2_1))
    myfile_valid_mse_dat_T2_1.write("%f \n" % (mse_T2_1))
    myfile_valid_mae_T2_1.write("%f \n" %(mae_T2_1))
        
    myfile_valid_psnr_T1_1.write("%f \n" % (psnr_T1_1))
    myfile_valid_mse_T1_1.write("%f \n" % (mse_T1_1))
    myfile_valid_psnr_dat_T1_1.write("%f \n" % (psnr_T1_1))
    myfile_valid_mse_dat_T1_1.write("%f \n" % (mse_T1_1))
    
    myfile_valid_psnr_T2_2.write("%f \n" % (psnr_T2_2))
    myfile_valid_mse_T2_2.write("%f \n" % (mse_T2_2))
    myfile_valid_psnr_dat_T2_2.write("%f \n" % (psnr_T2_2))
    myfile_valid_mse_dat_T2_2.write("%f \n" % (mse_T2_2))
    myfile_valid_mae_T2_2.write("%f \n" %(mae_T2_2))
        
    myfile_valid_psnr_T1_2.write("%f \n" % (psnr_T1_2))
    myfile_valid_mse_T1_2.write("%f \n" % (mse_T1_2))
    myfile_valid_psnr_dat_T1_2.write("%f \n" % (psnr_T1_2))
    myfile_valid_mse_dat_T1_2.write("%f \n" % (mse_T1_2))
    
    
    #scipy.misc.imsave('results_test/' +  'decoded.jpg', 255*decoded_imgs[0,:,:,0])
    valid_all_imgs_mse = np.mean((validation_T2 - decoded_imgs) ** 2)
    valid_all_imgs_mae = np.mean(np.abs(validation_T2 - decoded_imgs))
    check = math.sqrt(valid_all_imgs_mse)
    valid_all_imgs_psnr = 20 * math.log10( 1.0 / check)
    print("mse of all images in validation set is")
    print(valid_all_imgs_mse)
    print("mae of all images in validation set is")
    print(valid_all_imgs_mae)
    print("psnr of all images in validation set is")
    print(valid_all_imgs_psnr)
    
    valid_all_imgs_mse_file.write("%f \n" % valid_all_imgs_mse)
    valid_all_imgs_mae_file.write("%f \n" % valid_all_imgs_mae)
    valid_all_imgs_psnr_file.write("%f \n" % valid_all_imgs_psnr)
 #decoded is predicted images
    #if jj % 5 == 0:
    temp = np.zeros([resizeTo*3,resizeTo*3])  #zeros
    temp[:resizeTo,:resizeTo] = validation_T1[0,:,:,0]   #full image 
    temp[resizeTo:resizeTo*2,:resizeTo] = validation_T1[0,:,:,1]  #
    temp[resizeTo*2:,:resizeTo] = validation_T1[0,:,:,2]
    temp[:resizeTo,resizeTo:resizeTo*2] = validation_T2[0,:,:,0]
    temp[resizeTo:resizeTo*2,resizeTo:resizeTo*2] = validation_T2[0,:,:,1]
    temp[resizeTo*2:,resizeTo:resizeTo*2] = validation_T2[0,:,:,2]
    temp[:resizeTo,2*resizeTo:] = decoded_imgs[0,:,:,0]
    temp[resizeTo:resizeTo*2,2*resizeTo:] = decoded_imgs[0,:,:,1]
    temp[resizeTo*2:,2*resizeTo:] = decoded_imgs[0,:,:,2]
    temp = temp * 480
        #scipy.misc.imsave('results_sliced_4layers_custom_loss/' + str(jj) + '.jpg', temp)
        
    imageio.imwrite('/content/AllResults_HGSR/recon_epoch' + str(jj) + '.jpg', temp.astype(np.uint8))
        
    #jj +=1

    myfile_valid_psnr_T2_0.close()
    myfile_valid_mse_T2_0.close()
    myfile_valid_psnr_dat_T2_0.close()
    myfile_valid_mse_dat_T2_0.close()
    myfile_valid_mae_T2_0.close()

    myfile_valid_psnr_T1_0.close()
    myfile_valid_mse_T1_0.close()
    myfile_valid_psnr_dat_T1_0.close()
    myfile_valid_mse_dat_T1_0.close()

    
    myfile_valid_psnr_T2_1.close()
    myfile_valid_mse_T2_1.close()
    myfile_valid_psnr_dat_T2_1.close()
    myfile_valid_mse_dat_T2_1.close()
    myfile_valid_mae_T2_1.close()

    myfile_valid_psnr_T1_1.close()
    myfile_valid_mse_T1_1.close()
    myfile_valid_psnr_dat_T1_1.close()
    myfile_valid_mse_dat_T1_1.close()
    
    
    myfile_valid_psnr_T2_2.close()
    myfile_valid_mse_T2_2.close()
    myfile_valid_psnr_dat_T2_2.close()
    myfile_valid_mse_dat_T2_2.close()
    myfile_valid_mae_T2_2.close()

    myfile_valid_psnr_T1_2.close()
    myfile_valid_mse_T1_2.close()
    myfile_valid_psnr_dat_T1_2.close()
    myfile_valid_mse_dat_T1_2.close()

    valid_all_imgs_mse_file.close()
    valid_all_imgs_mae_file.close()
    valid_all_imgs_psnr_file.close()
    batch_loss_file.close()

test(model)
model.save('colabmodel.h5')

(480, 480, 18)
(480, 480, 18)
(480, 480, 18)
(480, 480, 18)
(480, 480, 18)
(480, 480, 18)
(480, 480, 18)
(480, 480, 18)
(480, 480, 18)
read both ground and input
made all the augmented images
(54, 480, 480, 3)
printing label shape below
(54, 480, 480, 3)
Training and Testing data has now following shapes
(43, 480, 480, 3)
(11, 480, 480, 3)
Training data lenth is :  43
Running epoch : 0
The shape is
(11, 480, 480, 3)
(11, 480, 480)
mse of all images in validation set is
0.0033461900583498473
mae of all images in validation set is
0.021748229466173084
psnr of all images in validation set is
24.754493954272924
Running epoch : 1
The shape is
(11, 480, 480, 3)
(11, 480, 480)
mse of all images in validation set is
0.003310181953966352
mae of all images in validation set is
0.02232217836765719
psnr of all images in validation set is
24.801481332841966
Running epoch : 2
The shape is
(11, 480, 480, 3)
(11, 480, 480)
mse of all images in validation set is
0.0029937597586979013
mae of all images 